# Fingerprint Matching using OpenCV's SIFT and FLANN

## 1. Import Necessary Libraries
First, ensure that you import the necessary libraries.

In [1]:
import cv2
import numpy as np
import os
from multiprocessing import Pool, cpu_count

## 2. Load the Fingerprint Image and Create SIFT Descriptor
You will load the fingerprint image and create the SIFT descriptor outside of the multiprocessing function. This is important because global objects like images can cause issues when pickling.

In [2]:
# Load the test fingerprint image
fingerprint_test = cv2.imread("TEST_1.tif")

# SIFT detector for the test image
sift = cv2.xfeatures2d.SIFT_create()
keypoints_1, descriptors_1 = sift.detectAndCompute(fingerprint_test, None)


[ WARN:0@0.033] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


## 3. Define the Functions
Define the functions for matching fingerprints, serializing matches, and processing results. Make sure these functions are defined at the top level (not inside another function) to ensure they can be pickled properly.

In [3]:
# Function to serialize DMatch objects (convert them to tuples)
def serialize_match(matches):
    return [(m.queryIdx, m.trainIdx, m.distance) for m in matches]

# Function to match a single fingerprint from the database
def match_fingerprint(file):
    fingerprint_database_image = cv2.imread("./database/" + file)
    if fingerprint_database_image is None:
        return None

    # Detect keypoints and compute descriptors for the database image
    keypoints_2, descriptors_2 = sift.detectAndCompute(fingerprint_database_image, None)

    if descriptors_1 is None or descriptors_2 is None:
        return None

    # Match descriptors using FLANN-based matcher
    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=10), dict())
    matches = flann.knnMatch(descriptors_1, descriptors_2, k=2)

    # Filter good matches
    match_points = []
    for p, q in matches:
        if p.distance < 0.1 * q.distance:
            match_points.append(p)

    keypoints = min(len(keypoints_1), len(keypoints_2))
    if keypoints == 0:
        return None

    threshold = 0.3
    match_ratio = len(match_points) / keypoints
    if match_ratio > threshold:
        # Serialize match points for pickling
        serialized_match_points = serialize_match(match_points)
        
        # If a match is found, return relevant information
        return {
            "file": file,
            "match_ratio": match_ratio,
            "match_points": serialized_match_points,  # Serialized match points
            "keypoints_2_len": len(keypoints_2)  # Return keypoints length for simplicity
        }
    return None

# Function to deserialize match points and draw matches
def deserialize_and_draw(fingerprint_test, keypoints_1, fingerprint_database_image, keypoints_2_len, match_points):
    # Create dummy keypoints since DMatch needs keypoint objects for visualization
    keypoints_2 = [cv2.KeyPoint() for _ in range(keypoints_2_len)]

    # Deserialize match points
    deserialized_matches = [cv2.DMatch(int(m[0]), int(m[1]), float(m[2])) for m in match_points]

    # Draw matches
    result_image = cv2.drawMatches(fingerprint_test, keypoints_1, 
                                   fingerprint_database_image, keypoints_2, 
                                   deserialized_matches, None)
    return result_image

# Function to process the results
def process_results(results):
    for result in results:
        if result is not None:
            print(f"% match: {result['match_ratio'] * 100}")
            print(f"Fingerprint ID: {result['file']}")
            
            # Load the fingerprint database image again for drawing
            fingerprint_database_image = cv2.imread("./database/" + result['file'])

            # Deserialize and draw the matches
            result_image = deserialize_and_draw(fingerprint_test, keypoints_1, fingerprint_database_image, 
                                                result['keypoints_2_len'], result['match_points'])
            result_image = cv2.resize(result_image, None, fx=2.5, fy=2.5)
            
            # Display the result using matplotlib
            # Uncomment below to display the result
            # plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
            # plt.axis('off')
            # plt.show()
            return True
    return False


## 4. Execute the Matching Process
Finally, run the multiprocessing logic. You should encapsulate this logic in a `main` guard to avoid issues with Jupyter's execution model.

In [4]:
# Get the list of files in the database
files = [file for file in os.listdir("database")]

# Create a pool of worker processes
with Pool(cpu_count()) as pool:
    # Distribute fingerprint matching across processes
    results = pool.map(match_fingerprint, files)

# Process and display the results
if not process_results(results):
    print("No matching fingerprint found in the database.")

% match: 89.65517241379311
Fingerprint ID: 106_3.tif
